In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from time import time
import pickle
import os
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import accuracy_score

In [2]:
df1 = pd.read_csv("C:/Users/CE206-26/Desktop/크롤링구글맵스라벨링1.csv",encoding='CP949')
df2 = pd.read_csv("C:/Users/CE206-26/Desktop/구글맵스띄어쓰기삭제.csv",encoding='CP949')

In [3]:
df1['라벨링'] = df1['라벨링'].astype('str') ### 라벨이 int면 라벨이라고 인식을 못함###

In [ ]:
###이거나###

df_train = df1
df_test = df2

In [49]:
###이거###

df_train = df1[330:]
df_test = df1[331:]

In [5]:
df_test['라벨링'] = '0' ### test 데이터로 쓰기 위해서는 라벨링 필요 ###

In [8]:
df1['라벨링'].dtypes ### dtype('O') 여야함###

dtype('O')

In [50]:
twitter = Twitter()
def tokenizer_twitter_morphs(doc):
    return twitter.morphs(doc)

def tokenizer_twitter_noun(doc):
    return twitter.nouns(doc)

def tokenizer_twitter_pos(doc):
    return twitter.pos(doc, norm=True, stem=True)

C:\Users\CE206-26\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [51]:
komoran = Komoran()
def tokenizer_noun(doc):
    return komoran.nouns(doc)

def tokenizer_morphs(doc):
    return komoran.morphs(doc)

In [52]:
### 리뷰들 토큰화 시간 오래걸림 ###

df_train['token_review'] = df_train['리뷰'].apply(tokenizer_twitter_morphs)
df_test['X_test_tokkened'] = df_test['리뷰'].apply(tokenizer_twitter_morphs)
tokens = [ t for d in df_train['token_review'] for t in d]

C:\Users\CE206-26\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\CE206-26\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [53]:
import nltk
text = nltk.Text(tokens, name='NMSC')

In [54]:
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, c) for d, c in df_train[['token_review', '라벨링']].values]
tagged_test_docs = [TaggedDocument(d, c) for d, c in df_test[['X_test_tokkened','라벨링']].values]

In [55]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

4

In [56]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [57]:
from gensim.models import Doc2Vec

In [58]:
doc_vectorizer = Doc2Vec()

In [59]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

2019-08-22 15:28:48,400 : INFO : collecting all words and their counts
2019-08-22 15:28:48,401 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-08-22 15:28:48,451 : INFO : collected 16418 word types and 6 unique tags from a corpus of 157 examples and 136053 words
2019-08-22 15:28:48,451 : INFO : Loading a fresh vocabulary
2019-08-22 15:28:48,473 : INFO : effective_min_count=5 retains 3168 unique words (19% of original 16418, drops 13250)
2019-08-22 15:28:48,474 : INFO : effective_min_count=5 leaves 115978 word corpus (85% of original 136053, drops 20075)
2019-08-22 15:28:48,488 : INFO : deleting the raw counts dictionary of 16418 items
2019-08-22 15:28:48,489 : INFO : sample=0.001 downsamples 43 most-common words
2019-08-22 15:28:48,490 : INFO : downsampling leaves estimated 89686 word corpus (77.3% of prior 115978)
2019-08-22 15:28:48,508 : INFO : estimated required memory for 3168 words and 100 dimensions: 4122000 bytes
2019-08-22 15:28:48,510 : IN

Doc2Vec(dm/m,d100,n5,w5,mc5,s0.001,t3)


In [60]:
doc_vectorizer.corpus_count

157

In [61]:
doc_vectorizer.iter

C:\Users\CE206-26\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [62]:
start = time()
for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time()
print("During Time: {}".format(end-start))

C:\Users\CE206-26\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until
2019-08-22 15:28:50,809 : INFO : training model with 3 workers on 3168 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-08-22 15:28:50,920 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 15:28:50,922 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-22 15:28:50,924 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-22 15:28:50,925 : INFO : EPOCH - 1 : training on 136053 raw words (89831 effective words) took 0.1s, 802392 effective words/s
2019-08-22 15:28:51,034 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 15:28:51,036 : INFO : worker thread finished; awaiting finish of 1 more threads


During Time: 6.024529218673706


In [63]:
model_name = 'Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model'
doc_vectorizer.save(model_name)

2019-08-22 15:28:59,159 : INFO : saving Doc2Vec object under Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model, separately None
2019-08-22 15:28:59,203 : INFO : saved Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model


In [64]:
doc_vectorizer = Doc2Vec.load(model_name)

2019-08-22 15:28:59,845 : INFO : loading Doc2Vec object from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model
2019-08-22 15:28:59,873 : INFO : loading vocabulary recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.vocabulary.* with mmap=None
2019-08-22 15:28:59,875 : INFO : loading trainables recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.trainables.* with mmap=None
2019-08-22 15:28:59,875 : INFO : loading wv recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.wv.* with mmap=None
2019-08-22 15:28:59,876 : INFO : loading docvecs recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.docvecs.* with mmap=None
2019-08-22 15:28:59,877 : INFO : loaded Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model


In [65]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags for doc in tagged_train_docs]

In [66]:
X_test = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]

In [67]:
y_test = [doc.tags for doc in tagged_test_docs]

In [68]:
print(len(X_train_np), len(y_train_np))

330 330


In [69]:
X_train_np = np.asarray(X_train)
X_test_np = np.array(X_test)

In [70]:
y_train_np = np.asarray(y_train)
y_test_np = np.array(y_test)

In [71]:
y_train_np = np.eye(7)[y_train_np.astype('int').reshape(-1)]
y_test_np = np.eye(7)[y_test_np.astype('int').reshape(-1)]

In [72]:
from sklearn.neighbors import KNeighborsClassifier

In [73]:
ml = KNeighborsClassifier(n_neighbors = 6, p=2, metric='minkowski')

In [74]:
ml.fit(X_train_np,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

In [75]:
y_pred = ml.predict(X_test)

In [76]:
y_pred

array(['4', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '1', '5',
       '1', '1', '1', '6', '1', '1', '1', '2', '1', '1', '1', '1', '1',
       '6', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '6', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '2', '1', '6', '1', '1', '1', '6', '1', '1',
       '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '1', '4', '4',
       '2', '6', '6', '6', '6', '6', '2', '6', '1', '3', '3', '6', '3',
       '6', '3', '1', '6', '4', '1', '6', '6', '1', '1', '1', '1', '2',
       '1', '1', '6', '1', '1', '2', '1', '1', '4', '3', '1', '6', '1',
       '1', '1', '6', '4', '1', '3', '3', '1', '3', '3', '3', '1', '1',
       '1', '1', '6', '1', '6', '6', '1', '1', '1', '1', '1', '6', '1'],
      dtype='<U1')

In [77]:
print('총 테스트 개수:%d, 오류개수:%d' %(len(y_test), (y_test != y_pred).sum()))
print('정확도: %.3f' %accuracy_score(y_test,y_pred))

총 테스트 개수:156, 오류개수:37
정확도: 0.763
